In [1]:
import numpy as np
import scipy as sp
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

In [2]:
from ema_workbench import (Model, CategoricalParameter,
                           ScalarOutcome, IntegerParameter, RealParameter,Constant)
from ema_workbench.em_framework.optimization import (HyperVolume, EpsilonProgress)
from dike_model_function import DikeNetwork  # @UnresolvedImport

def sum_over(*args):
    return sum(args)

In [3]:
from ema_workbench import (Model, RealParameter, ScalarOutcome,
                           MultiprocessingEvaluator, ema_logging,
                           Constant, Policy, Scenario)

ema_logging.log_to_stderr(ema_logging.INFO)

from ema_workbench.em_framework.evaluators import perform_experiments
from ema_workbench.em_framework.samplers import sample_uncertainties
from ema_workbench import ScalarOutcome
from ema_workbench.util import ema_logging
import time
from problem_formulation import get_model_for_problem_formulation

ema_logging.log_to_stderr(ema_logging.INFO)

dike_model, planning_steps = get_model_for_problem_formulation(3)

In [4]:
#for unc in dike_model.uncertainties:
 #   print(repr(unc))

In [5]:
#for unc in dike_model.uncertainties:
#    print(repr(unc))
    
uncertainties = dike_model.uncertainties

import copy
uncertainties = copy.deepcopy(dike_model.uncertainties)

In [6]:
#for policy in dike_model.levers:
#    print(repr(policy))
    
levers = dike_model.levers 

import copy
levers = copy.deepcopy(dike_model.levers)

In [7]:
#for outcome in dike_model.outcomes:
#    print(repr(outcome))

In [9]:
def problem_formulation_actor(problem_formulation_actor):   
    # Load the model:
    function = DikeNetwork()
    # workbench model:
    model = Model('dikesnet', function=function)
    # Outcomes are all costs, thus they have to minimized:
    direction = ScalarOutcome.MINIMIZE
        
    model.levers = levers

    if problem_formulation_actor == 4: # RWS
        model.outcomes.clear()
        model.outcomes = [
            ScalarOutcome('Expected Annual Damage',
                            variable_name=['{}_Expected Annual Damage {}'.format(dike, steps) for dike in function.dikelist for steps in function.planning_steps],
                            function=sum_over, kind=direction, expected_range=(0,1000000000)),

            ScalarOutcome('Total Investment Costs',
                            variable_name=['{}_Dike Investment Costs {}'.format(dike, steps) for dike in function.dikelist for steps in function.planning_steps]+
                          ['RfR Total Costs {}'.format(steps) for steps in function.planning_steps]+
                          ['Expected Evacuation Costs {}'.format(steps) for steps in function.planning_steps],
                            function=sum_over, kind=direction, expected_range=(0,5000000000)),

            ScalarOutcome('Expected Number of Deaths',
                            variable_name=['{}_Expected Number of Deaths {}'.format(dike, steps) for dike in function.dikelist for steps in function.planning_steps],
                            function=sum_over, kind=direction, expected_range=(0,10))] 
    
    elif problem_formulation_actor == 5: # GELDERLAND
        model.outcomes.clear()
        model.outcomes = [
            ScalarOutcome('Expected Annual Damage A1-4',
                            variable_name=['{}_Expected Annual Damage {}'.format(dike, steps) for dike in function.dikelist[:-1] for steps in function.planning_steps], 
                          function=sum_over, kind=direction),

            ScalarOutcome('Investment Costs A1-4',
                            variable_name=['{}_Dike Investment Costs {}'.format(dike, steps) for dike in function.dikelist[:-1] for steps in function.planning_steps], 
                          function=sum_over, kind=direction),

            ScalarOutcome('Expected Number of Deaths in A1-4',
                            variable_name=['{}_Expected Number of Deaths {}'.format(dike, steps) for dike in function.dikelist[:-1] for steps in function.planning_steps],
                          function=sum_over, kind=direction)]

    elif problem_formulation_actor == 6: # OVERIJSSEL
        model.outcomes.clear()
        model.outcomes = [
            ScalarOutcome('Expected Annual Damage A5', variable_name=['A.5_Expected Annual Damage {}'.format(steps) for steps in function.planning_steps], function=sum_over, kind=direction),

            ScalarOutcome('Investment Costs A5', variable_name=['A.5_Dike Investment Costs {}'.format(steps) for steps in function.planning_steps], function=sum_over, kind=direction),

            ScalarOutcome('Expected Number of Deaths in A5', variable_name=['A.5_Expected Number of Deaths {}'.format(steps) for steps in function.planning_steps], function=sum_over, kind=direction)]
    
    else:
        raise TypeError('unknown identifier')
    return model

In [10]:
model = problem_formulation_actor(4) #this is running the model for Rijkswaterstaat
#If you want to run the model for gelderland --> 5, Overrijsel --> 6
#For the dike rings, there objective functions need to be implemented (maybe to much work?)

In [ ]:
#specify metrics for the MOEA analysis 
convergence_metrics = [HyperVolume.from_outcomes(model.outcomes), EpsilonProgress()]

#Run the MOEA analysis with convergence statistics
with MultiprocessingEvaluator(model) as evaluator:
    results, convergence = evaluator.optimize(nfe=10000, searchover='levers',
                                    epsilons=[0.5, 0.5, 0.5],
                                    convergence=convergence_metrics)

[MainProcess/INFO] pool started
[MainProcess/INFO] generation 0: 0/10000 nfe


In [ ]:
#Plot the convergence statistics
fig, (ax1, ax2) = plt.subplots(ncols=2, sharex=True, figsize=(8,4))
ax1.plot(convergence.nfe, convergence.epsilon_progress)
ax1.set_ylabel('$\epsilon$-progress')
ax2.plot(convergence.nfe, convergence.hypervolume)
ax2.set_ylabel('hypervolume')

ax1.set_xlabel('number of function evaluations')
ax2.set_xlabel('number of function evaluations')
plt.show()

In [ ]:
#when we achieve convergence, check the results
#3d scatterplot for pareto optimal solutions
from mpl_toolkits.mplot3d import Axes3D  

outcomes = results.loc[:, ['Expected Annual Damage', 'Total Investment Costs', 'Expected Number of Deaths']]

fig = plt.figure(figsize=(8,8))
ax = fig.add_subplot(111, projection='3d')
ax.scatter(outcomes.max_P, outcomes.utility, outcomes.reliability)
ax.set_xlabel('Expected Annual Damage')
ax.set_ylabel('Total Investment Costs')
ax.set_zlabel('Expected Number of Deaths')
plt.show()

In [ ]:
#evaluate trade offs between the three objectives
from ema_workbench.analysis import parcoords
limits = parcoords.get_limits(outcomes)
axes = parcoords.ParallelAxes(limits)
axes.plot(outcomes)

plt.show()
